# Read tables from PDF

In this notebook I'm going to extract the table containing our data from the PDF report (downloaded using our python scripts in the previous step) and clean it up using pandas. By the end of the notebook, I want to have a clean dataset that can be analyzed.
 
I'm using a notebook to explore what steps are needed to get to the end result for 1 PDF report. I often find it easier and quicker to do this kind of work in a notebook, as opposed to doing it in a python script and/or interpreter. Once I have a solution, I can generalize it to handle multiple reports.

**Contents**
1. Read PDF
2. Clean data

In [1]:
import re
from pathlib import Path

import pandas as pd
import tabula

pd.set_option('display.max_columns', 25)

In [2]:
project_folder = Path.cwd().parent
data_folder = project_folder / 'data'
raw_data_folder = data_folder / 'raw'
processed_data_folder = data_folder / 'processed'

## 1. Read PDF

Reading tables from a PDF is relatively straightforward using tabula. It's a simple one-liner in fact. The main challenge is to find a configuration that gives you the cleanest results. You'll have to [dig into the documentation](https://tabula-py.readthedocs.io/en/latest/tabula.html) to find out what options tabula gives you.

Although not essential, I'm going to specify which pages tabula should limit to, so that extracting the tables is more efficient. Once tabula has read the PDF, it conveniently gives us a pandas DataFrame, so that we can get straight into the cleaning part.

In [3]:
pdf_name = '20200330-sitrep-70-covid-19.pdf'
pages = '2-6'

In [4]:
# check that tabula is installed
tabula.environment_info()

Python version:
    3.7.7 (default, Mar 23 2020, 23:19:08) [MSC v.1916 64 bit (AMD64)]
Java version:
    java version "1.8.0_241"
Java(TM) SE Runtime Environment (build 1.8.0_241-b07)
Java HotSpot(TM) Client VM (build 25.241-b07, mixed mode)
tabula-py version: 2.1.0
platform: Windows-10-10.0.17763-SP0
uname:
    uname_result(system='Windows', node='AU16920', release='10', version='10.0.17763', machine='AMD64', processor='Intel64 Family 6 Model 158 Stepping 13, GenuineIntel')
linux_distribution: ('MSYS_NT-10.0-17763', '3.0.7', '')
mac_ver: ('', ('', '', ''), '')
    


In [5]:
# lattice=True gives cleaner results for our use case
# we're passing the option header=None to pandas so that we can clean up the header names first before setting them
tables = tabula.read_pdf(raw_data_folder / pdf_name, pages=pages, lattice=True, pandas_options={'header': None})

Got stderr: Apr 27, 2020 8:16:15 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font BCDGEE+Calibri-Bold are not implemented in PDFBox and will be ignored
Apr 27, 2020 8:16:16 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font BCDHEE+Calibri are not implemented in PDFBox and will be ignored



In [6]:
len(tables)

18

In [7]:
# number of rows in each table 
for i, table in enumerate(tables):
    print(f"{i}: {len(table)}")

0: 47
1: 3
2: 4
3: 4
4: 3
5: 4
6: 3
7: 3
8: 53
9: 49
10: 50
11: 26
12: 2
13: 2
14: 2
15: 2
16: 2
17: 2


tabula has returned 18 tables, even though we only had 5 pages (1 table per page in the PDF). Let's inspect what is in all these tables.

In [8]:
# first table has 47 rows, so we would expect it to have our data
tables[0].head(10)

# we can see it has countries starting at index 7

,0,1,2,3,4,5,6
0,Reporting Country/\rTerritory/Area†,Total\rconfirmed\r‡ cases,Total\rconfirmed\rnew cases,Total\rdeaths,Total\rnew\rdeaths,Transmission\rclassification§,Days since last\rreported case
1,NaN,Total,Total,Total,NaN,NaN,NaN
2,NaN,eporting Country/,Total,Transmission,Days since last,NaN,NaN
3,NaN,confirmed,confirmed,new,NaN,NaN,NaN
4,NaN,erritory/Area†,deaths,classification§,reported case,NaN,NaN
5,NaN,‡ cases,new cases,deaths,NaN,NaN,NaN
6,NaN,estern Pacific Region,NaN,NaN,NaN,NaN,NaN
7,China,82447,106,3310,4,Local transmission,0
8,Republic of Korea,9661,78,158,6,Local transmission,0
9,Australia,3966,0,16,0,Local transmission,1


In [9]:
# looking at the last rows, we can check that it aligns with the table on the first page
tables[0].tail(10)

# Poland is indeed the last country on page 2 in the PDF

,0,1,2,3,4,5,6
37,Austria,8813,522,86,18,Local transmission,0
38,Portugal,5962,792,119,19,Local transmission,0
39,Israel,4247,382,15,0,Local transmission,0
40,Norway,4102,257,22,2,Local transmission,0
41,Sweden,3700,253,110,8,Local transmission,0
42,Czechia,2829,166,16,5,Local transmission,0
43,Ireland,2615,200,46,10,Local transmission,0
44,Denmark,2395,194,72,7,Local transmission,0
45,Luxembourg,1950,119,21,3,Local transmission,0
46,Poland,1862,224,22,4,Local transmission,0


In [10]:
# inspect a table with only a handful rows
tables[1]

,0
0,Reporting Country/\rTerritory/Area†
1,eporting Country/
2,erritory/Area†


In [11]:
# let's pick another one
tables[5]

,0
0,Total\rnew\rdeaths
1,Total
2,new
3,deaths


In [12]:
# 2nd table that has more than a handful rows
tables[8].head()

# Romania is the first country on page 3 in the PDF

,0,1,2,3,4,5,6
0,Romania,1760,308.0,40.0,11.0,Local transmission,0.0
1,Russian Federation,1534,0.0,10.0,2.0,Local transmission,1.0
2,Finland,1218,0.0,11.0,2.0,Local transmission,1.0
3,Greece,1156,95.0,38.0,6.0,Local transmission,0.0
4,Iceland,1020,57.0,2.0,0.0,Local transmission,0.0


**Observations**  
- tabula has returned 18 tables, even though we only read 5 pages from the PDF (with 1 table per page)
- thus far it seems that the tables with more than a handful rows are the tables that contain our data; the rest of the tables seem to be just noise
- something strange is going on with the header names in the first table
- ... but fortunately, the first row has full header names that are almost clean
- the relevant tables with data  seem clean at first glance
- the index starts at 0 in every table. Albeit minor, we may want to reset it after combining tables so that we have a unique index.

**Next steps**  
For cleaning up our data, we want to start off with:
1. cleaning up header names 
2. combining all relevant tables into one
3. resetting the index

## 2. Clean data

In [13]:
# raw header names
header_names = tables[0].iloc[0,:].values
header_names

array(['Reporting Country/\rTerritory/Area†', 'Total\rconfirmed\r‡ cases',
       'Total\rconfirmed\rnew cases', 'Total\rdeaths',
       'Total\rnew\rdeaths', 'Transmission\rclassification§',
       'Days since last\rreported case'], dtype=object)

In [14]:
clean_names = [s.replace('\r', ' ') for s in header_names]
clean_names

['Reporting Country/ Territory/Area†',
 'Total confirmed ‡ cases',
 'Total confirmed new cases',
 'Total deaths',
 'Total new deaths',
 'Transmission classification§',
 'Days since last reported case']

In [15]:
# remove symbols
p = re.compile(r'[^\w\s\/]+')
clean_names = [re.sub(p, '', s) for s in clean_names]
clean_names

['Reporting Country/ Territory/Area',
 'Total confirmed  cases',
 'Total confirmed new cases',
 'Total deaths',
 'Total new deaths',
 'Transmission classification',
 'Days since last reported case']

In [16]:
# combine tables into one DataFrame
df = pd.concat(tables)
df.shape

(261, 21)

In [17]:
df = df.reset_index().drop(labels='index', axis=1)

In [18]:
# before removing the extra columns (7-20), check what is in there
df.iloc[:, 7:].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   7       3 non-null      float64
 1   8       0 non-null      float64
 2   9       0 non-null      float64
 3   10      3 non-null      float64
 4   11      0 non-null      float64
 5   12      0 non-null      float64
 6   13      3 non-null      float64
 7   14      0 non-null      float64
 8   15      0 non-null      float64
 9   16      2 non-null      object 
 10  17      0 non-null      float64
 11  18      0 non-null      float64
 12  19      2 non-null      float64
 13  20      0 non-null      float64
dtypes: float64(13), object(1)
memory usage: 28.7+ KB


In [19]:
# most columns are empty
# inspect those few columns with values
df[df.iloc[:, [7, 10, 13, 16, 19]].notna().any(axis=1)]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
242,NaN,éunion,NaN,NaN,207,NaN,NaN,64.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,Local transmission,NaN,NaN,0.0,NaN
243,NaN,ayotte,NaN,NaN,82,NaN,NaN,32.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,Local transmission,NaN,NaN,0.0,NaN
248,NaN,rand total,NaN,NaN,693282,NaN,NaN,58469.0,NaN,NaN,33106.0,NaN,NaN,3215.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Observations**  
- The last row is a totals row, which we can safely remove
- The other two rows are countries from the last page, but it has problems
    - The first letter of the country name is missing
    - The name appears in the second column rather than the first
    - The numbers are spread across various columns, rather than displayed in just the first 7 columns

For now, we take a note of this, and we will get back to this later. Let's remove all the extra columns.

In [20]:
# instead of removing the extra columns, we can also just keep the first 7 columns
df_7col = df.iloc[:,0:7].copy()
df_7col.head()

,0,1,2,3,4,5,6
0,Reporting Country/\rTerritory/Area†,Total\rconfirmed\r‡ cases,Total\rconfirmed\rnew cases,Total\rdeaths,Total\rnew\rdeaths,Transmission\rclassification§,Days since last\rreported case
1,NaN,Total,Total,Total,NaN,NaN,NaN
2,NaN,eporting Country/,Total,Transmission,Days since last,NaN,NaN
3,NaN,confirmed,confirmed,new,NaN,NaN,NaN
4,NaN,erritory/Area†,deaths,classification§,reported case,NaN,NaN


In [21]:
# set clean header names
df_7col.columns = clean_names
df_7col.head()

,Reporting Country/ Territory/Area,Total confirmed cases,Total confirmed new cases,Total deaths,Total new deaths,Transmission classification,Days since last reported case
0,Reporting Country/\rTerritory/Area†,Total\rconfirmed\r‡ cases,Total\rconfirmed\rnew cases,Total\rdeaths,Total\rnew\rdeaths,Transmission\rclassification§,Days since last\rreported case
1,NaN,Total,Total,Total,NaN,NaN,NaN
2,NaN,eporting Country/,Total,Transmission,Days since last,NaN,NaN
3,NaN,confirmed,confirmed,new,NaN,NaN,NaN
4,NaN,erritory/Area†,deaths,classification§,reported case,NaN,NaN


In [22]:
# remove empty rows
df_countries_only = df_7col[df_7col['Reporting Country/ Territory/Area'].notna()]
df_countries_only.shape

(235, 7)

In [23]:
df_countries_only.head()

,Reporting Country/ Territory/Area,Total confirmed cases,Total confirmed new cases,Total deaths,Total new deaths,Transmission classification,Days since last reported case
0,Reporting Country/\rTerritory/Area†,Total\rconfirmed\r‡ cases,Total\rconfirmed\rnew cases,Total\rdeaths,Total\rnew\rdeaths,Transmission\rclassification§,Days since last\rreported case
7,China,82447,106,3310,4,Local transmission,0
8,Republic of Korea,9661,78,158,6,Local transmission,0
9,Australia,3966,0,16,0,Local transmission,1
10,Malaysia,2470,150,34,7,Local transmission,0


In [24]:
# remove first row
df_countries_only = df_countries_only.iloc[1:, :]
df_countries_only.head()

,Reporting Country/ Territory/Area,Total confirmed cases,Total confirmed new cases,Total deaths,Total new deaths,Transmission classification,Days since last reported case
7,China,82447,106,3310,4,Local transmission,0
8,Republic of Korea,9661,78,158,6,Local transmission,0
9,Australia,3966,0,16,0,Local transmission,1
10,Malaysia,2470,150,34,7,Local transmission,0
11,Japan,1866,173,54,2,Local transmission,0


This is already starting to look like the output we want!

In [25]:
# let's see if we have more empty cells
df_countries_only.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234 entries, 7 to 256
Data columns (total 7 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Reporting Country/ Territory/Area  234 non-null    object
 1   Total confirmed  cases             201 non-null    object
 2   Total confirmed new cases          201 non-null    object
 3   Total deaths                       201 non-null    object
 4   Total new deaths                   201 non-null    object
 5   Transmission classification        201 non-null    object
 6   Days since last reported case      201 non-null    object
dtypes: object(7)
memory usage: 14.6+ KB


In [26]:
# remove rows that have no numbers
df_countries_only = df_countries_only[df_countries_only['Total confirmed  cases'].notna()]
df_countries_only.shape

(201, 7)

A good validation check is to compare the sum of each column with the totals row in the PDF. Let's check first what data types we have before we attempt calculating the sum.

In [27]:
df_countries_only.dtypes

Reporting Country/ Territory/Area    object
Total confirmed  cases               object
Total confirmed new cases            object
Total deaths                         object
Total new deaths                     object
Transmission classification          object
Days since last reported case        object
dtype: object

Everything is parsed as an `object`, which is usually used for strings or columns with mixed data types.

In [28]:
# trying to convert to int64 gives an error because of NaNs, so setting to float64 for now
# if you're on pandas >1.0, there is a new int datatype (Int64) that allows NaNs
df_countries_only = df_countries_only.astype(dtype={
    'Reporting Country/ Territory/Area': 'object',
    'Total confirmed  cases': 'float64',
    'Total confirmed new cases': 'float64',
    'Total deaths': 'float64',
    'Total new deaths': 'float64',
    'Transmission classification': 'object',
    'Days since last reported case': 'float64',
})
df_countries_only.dtypes

Reporting Country/ Territory/Area     object
Total confirmed  cases               float64
Total confirmed new cases            float64
Total deaths                         float64
Total new deaths                     float64
Transmission classification           object
Days since last reported case        float64
dtype: object

In [29]:
# now we can calculate the sum
df_countries_only.select_dtypes('float64').sum()

Total confirmed  cases           692993.0
Total confirmed new cases         58373.0
Total deaths                      33106.0
Total new deaths                   3215.0
Days since last reported case       188.0
dtype: float64

**Observations**  
Compared to the totals in the report: 
- "Total confirmed cases" is _lower_ 
- "Total confirmed new cases" is _lower_ 
- "Total deaths" is _equal_
- "Total new deaths" is _equal_
- "Days since last reported case" has no total in the report

Though not perfect yet, these results are promising! In fact, let's revisit our earlier note about the two missing countries and see if that explains the gap.


_From the PDF_

| Reporting Country/ Territory/Area | Total confirmed  cases | Total confirmed new cases | Total deaths | Total new deaths | Transmission classification | Days since last reported case |
| --------------------------------- | :--------------------: | :-----------------------: | :----------: | :--------------: | --------------------------- | :---------------------------: |
| Réunion                           |          207           |            64             |      0       |        0         | Local transmission          |               0               |
| Mayotte                           |           82           |            32             |      0       |        0         | Local transmission          |               0               |
| **Total**                         |        **289**         |          **96**           |    **0**     |      **0**       |                             |             **0**             |

Let's do the math:
- 692,993 + 289 = 693,282
- 58,373 + 96 = 58,469

BINGO!! This adds up to the totals reported in the PDF!!

**Next steps**  
We need to somehow add in the 2 missing countries. We could add it in manually, but that wouldn't be much fun. Let's try another method.

The first time we used the setting `lattice=True` because it gave us a cleaner output. We can try `lattice=False` and see if that gives us clean output for just the 2 missing countries. For all the remaining countries we can rely on the DataFrame we already have.

### Missing countries

In [30]:
# lattice=False is the default, so we can leave it out
# we can also pass in the clean header names
tables_lat_false = tabula.read_pdf(raw_data_folder / pdf_name, pages=pages, pandas_options={'names': clean_names})

Got stderr: Apr 27, 2020 8:16:19 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font BCDGEE+Calibri-Bold are not implemented in PDFBox and will be ignored
Apr 27, 2020 8:16:20 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font BCDGEE+Calibri-Bold are not implemented in PDFBox and will be ignored
Apr 27, 2020 8:16:22 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font BCDGEE+Calibri-Bold are not implemented in PDFBox and will be ignored
Apr 27, 2020 8:16:23 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font BCDHEE+Calibri are not implemented in PDFBox and will be ignored
Apr 27, 2020 8:16:23 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font BCDGEE+Calibri-Bold are not implemented in PDFBox and will be ignored
Apr 27, 2020 8:16:23 AM org.apache.pdfbox.pdmode

In [31]:
len(tables_lat_false)

5

In [32]:
for i, table in enumerate(tables_lat_false):
    print(f"{i}: {len(table)}")

0: 48
1: 53
2: 49
3: 50
4: 29


In [33]:
# inspect first table
tables_lat_false[0].head(10)

,Reporting Country/ Territory/Area,Total confirmed cases,Total confirmed new cases,Total deaths,Total new deaths,Transmission classification,Days since last reported case
0,NaN,Total,Total,NaN,Total,NaN,NaN
1,Reporting Country/,NaN,NaN,Total,NaN,Transmission,Days since last
2,Territory/Area †,confirmed ‡ cases,confirmed new cases,deaths,new deaths,classification§,reported case
3,Western Pacific Region,NaN,NaN,NaN,NaN,NaN,NaN
4,China,82447,106,3310,4,Local transmission,0
5,Republic of Korea,9661,78,158,6,Local transmission,0
6,Australia,3966,0,16,0,Local transmission,1
7,Malaysia,2470,150,34,7,Local transmission,0
8,Japan,1866,173,54,2,Local transmission,0
9,Philippines,1418,343,71,3,Local transmission,0


In [34]:
tables_lat_false[0].tail()

,Reporting Country/ Territory/Area,Total confirmed cases,Total confirmed new cases,Total deaths,Total new deaths,Transmission classification,Days since last reported case
43,Czechia,2829,166,16,5,Local transmission,0
44,Ireland,2615,200,46,10,Local transmission,0
45,Denmark,2395,194,72,7,Local transmission,0
46,Luxembourg,1950,119,21,3,Local transmission,0
47,Poland,1862,224,22,4,Local transmission,0


**Observations**
- We've got 5 tables, as we'd expect after reading 5 pages
- First table has some header issues in the first few rows
- Otherwise, the first table starts with China and ends with Poland, as expected (page 2 in PDF)
- Output is relatively clean

Having seen this output, you might wonder why we didn't use this approach from the start. I discovered an issue when I first tried it, and decided to change the approach. Let me show you the problems I found.

In [35]:
# some country names span two rows within a cell in the PDF
tables_lat_false[0].iloc[16:18, :]

,Reporting Country/ Territory/Area,Total confirmed cases,Total confirmed new cases,Total deaths,Total new deaths,Transmission classification,Days since last reported case
16,Lao People's,NaN,NaN,NaN,NaN,NaN,NaN
17,Democratic Republic,8,2,0,0,Local transmission,0


"Lao People's Democratic Republic" is one example where the country name spans two rows within a cell in the PDF.  When using `lattice=False` you end up with two rows in the DataFrame, one without any numbers. If this is the only row with NaNs, then it isn't too hard to solve. However, we also have the following rows.

In [36]:
# another example
tables_lat_false[0].iloc[28:32, :]

,Reporting Country/ Territory/Area,Total confirmed cases,Total confirmed new cases,Total deaths,Total new deaths,Transmission classification,Days since last reported case
28,European Region,NaN,NaN,NaN,NaN,NaN,NaN
29,Italy,97689,5217,10781,758,Local transmission,0
30,Spain,78797,6549,6528,838,Local transmission,0
31,Germany,57298,4751,455,66,Local transmission,0


The pattern for "European Region" is exactly the same. However, this is a subsection title and not a country. So we want to remove this row, while keeping/combining the rows for "Lao People's Democratic Republic". Since both rows are nearly indistinguishable, there is no way to build a generic rule that can deal with this.

There are several options for dealing with such problems. I've decided to do the following:
- Create the first DataFrame using `lattice=True`, which becomes the master table (already done in first section)
- Read the table using `lattice=False` and append only countries to the master table that are missing

In [37]:
# first check if the missing countries are clean in the lattice=False table
tables_lat_false[-1].tail(10)

# we see them at index 20-21 and the data is clean!

,Reporting Country/ Territory/Area,Total confirmed cases,Total confirmed new cases,Total deaths,Total new deaths,Transmission classification,Days since last reported case
19,Territories**,NaN,NaN,NaN,NaN,NaN,NaN
20,Réunion,207.0,64.0,0.0,0.0,Local transmission,0.0
21,Mayotte,82.0,32.0,0.0,0.0,Local transmission,0.0
22,Subtotal for all,NaN,NaN,NaN,NaN,NaN,NaN
23,NaN,692570.0,58469.0,33099.0,3215.0,NaN,NaN
24,regions,NaN,NaN,NaN,NaN,NaN,NaN
25,International,NaN,NaN,NaN,NaN,NaN,NaN
26,conveyance (Diamond,712.0,0.0,7.0,0.0,Local transmission,14.0
27,Princess),NaN,NaN,NaN,NaN,NaN,NaN
28,Grand total,693282.0,58469.0,33106.0,3215.0,NaN,NaN


In [38]:
df_lat_false = pd.concat(tables_lat_false)
df_lat_false.shape

(229, 7)

In [39]:
# remove all rows that don't have numbers (e.g. subsection titles)
df_lat_false = df_lat_false.dropna(how='any')
df_lat_false.shape

(204, 7)

In [40]:
df_lat_false = df_lat_false.reset_index().drop(labels='index', axis=1)
df_lat_false.head()

,Reporting Country/ Territory/Area,Total confirmed cases,Total confirmed new cases,Total deaths,Total new deaths,Transmission classification,Days since last reported case
0,Territory/Area †,confirmed ‡ cases,confirmed new cases,deaths,new deaths,classification§,reported case
1,China,82447,106,3310,4,Local transmission,0
2,Republic of Korea,9661,78,158,6,Local transmission,0
3,Australia,3966,0,16,0,Local transmission,1
4,Malaysia,2470,150,34,7,Local transmission,0


In [41]:
# remove duplication of the header in the first row
df_lat_false = df_lat_false.iloc[1:, :]
df_lat_false.head()

,Reporting Country/ Territory/Area,Total confirmed cases,Total confirmed new cases,Total deaths,Total new deaths,Transmission classification,Days since last reported case
1,China,82447,106,3310,4,Local transmission,0
2,Republic of Korea,9661,78,158,6,Local transmission,0
3,Australia,3966,0,16,0,Local transmission,1
4,Malaysia,2470,150,34,7,Local transmission,0
5,Japan,1866,173,54,2,Local transmission,0


In [42]:
df_lat_false.dtypes

Reporting Country/ Territory/Area    object
Total confirmed  cases               object
Total confirmed new cases            object
Total deaths                         object
Total new deaths                     object
Transmission classification          object
Days since last reported case        object
dtype: object

In [43]:
# same issue as we had previously
df_lat_false = df_lat_false.astype(dtype={
    'Reporting Country/ Territory/Area': 'object',
    'Total confirmed  cases': 'float64',
    'Total confirmed new cases': 'float64',
    'Total deaths': 'float64',
    'Total new deaths': 'float64',
    'Transmission classification': 'object',
    'Days since last reported case': 'float64',
})
df_lat_false.dtypes

Reporting Country/ Territory/Area     object
Total confirmed  cases               float64
Total confirmed new cases            float64
Total deaths                         float64
Total new deaths                     float64
Transmission classification           object
Days since last reported case        float64
dtype: object

In [44]:
df_lat_false.select_dtypes('float64').sum()

Total confirmed  cases           693282.0
Total confirmed new cases         58469.0
Total deaths                      33106.0
Total new deaths                   3215.0
Days since last reported case       188.0
dtype: float64

**Observations**  
Compared to the totals in the report:

- "Total confirmed cases" is _equal_
- "Total confirmed new cases" is _equal_
- "Total deaths" is _equal_
- "Total new deaths" is _equal_
- "Days since last reported case" has no total in the report

That's awesome! We already have the correct totals!

**Next steps**  
I've tried several methods to get the desired results, and found the following the easiest. We know we have problems with slight differences in the country names between both tables. So, to identify countries in the secondary table that are missing in our master table, we can
- first combine the 2 tables into 1 DataFrame
- remove duplicate rows to get rid of exact matches
- remove duplicate rows a second time, but this time only based on the data columns (i.e. excluding country name)

Finally, we can append the countries that are left to the master table.

In [45]:
# combine master table and secondary table
df_combined = pd.concat([df_countries_only, df_lat_false])
df_combined.shape

(404, 7)

In [46]:
# get all headers except for country name
data_columns = df_combined.columns[1:]
data_columns

Index(['Total confirmed  cases', 'Total confirmed new cases', 'Total deaths',
       'Total new deaths', 'Transmission classification',
       'Days since last reported case'],
      dtype='object')

In [47]:
# keep countries that only exist in either the master table or the secondary table
df_unique_countries = df_combined.drop_duplicates(subset='Reporting Country/ Territory/Area', keep=False)
df_unique_countries.reset_index().drop(labels='index', axis=1)

,Reporting Country/ Territory/Area,Total confirmed cases,Total confirmed new cases,Total deaths,Total new deaths,Transmission classification,Days since last reported case
0,Lao People's\rDemocratic Republic,8.0,2.0,0.0,0.0,Local transmission,0.0
1,Northern Mariana\rIslands\r(Commonwealth of\rthe),2.0,0.0,0.0,0.0,Under investigation,1.0
2,Central African\rRepublic,6.0,0.0,0.0,0.0,Imported cases only,2.0
3,International\rconveyance (Diamond\rPrincess),712.0,0.0,7.0,0.0,Local transmission,14.0
4,Democratic Republic,8.0,2.0,0.0,0.0,Local transmission,0.0
5,the),2.0,0.0,0.0,0.0,Under investigation,1.0
6,Republic,6.0,0.0,0.0,0.0,Imported cases only,2.0
7,Réunion,207.0,64.0,0.0,0.0,Local transmission,0.0
8,Mayotte,82.0,32.0,0.0,0.0,Local transmission,0.0
9,conveyance (Diamond,712.0,0.0,7.0,0.0,Local transmission,14.0


**Observations**
- We see "Réunion" and "Mayotte", which are the 2 countries we're after
- The remaining rows are actually duplicates where the country name is spelled differently, e.g. index 0 and 4 are both referring to "Lao People's Democratic Republic"

**Next steps**  
We can again remove duplicates based on the numbers and Transmission classification (all columns excluding country name). This is not the most robust method and should be used with care. However, it gives us the desired result in this case. We can then append the 2 countries to our master table.

In [48]:
# getting the "actual" missing countries
df_missing_countries = df_unique_countries.drop_duplicates(subset=data_columns, keep=False)
df_missing_countries

,Reporting Country/ Territory/Area,Total confirmed cases,Total confirmed new cases,Total deaths,Total new deaths,Transmission classification,Days since last reported case
201,Réunion,207.0,64.0,0.0,0.0,Local transmission,0.0
202,Mayotte,82.0,32.0,0.0,0.0,Local transmission,0.0


In [49]:
# append missing countries
df_final = pd.concat([df_countries_only, df_missing_countries]).reset_index().drop(labels='index', axis=1)

In [50]:
# calculate sum for final check
df_final.select_dtypes('float64').sum()

Total confirmed  cases           693282.0
Total confirmed new cases         58469.0
Total deaths                      33106.0
Total new deaths                   3215.0
Days since last reported case       188.0
dtype: float64

And there we have it! The numbers are now identical to the totals in the PDF report.

In [51]:
# convert float to int for cleaner output
# this is now possible because we've cleaned up the data and removed all the NaNs
df_final = df_final.astype(dtype={
    'Reporting Country/ Territory/Area': 'string',
    'Total confirmed  cases': 'int64',
    'Total confirmed new cases': 'int64',
    'Total deaths': 'int64',
    'Total new deaths': 'int64',
    'Transmission classification': 'string',
    'Days since last reported case': 'int64',
})

In [52]:
# clean up \r from country names
df_final['Reporting Country/ Territory/Area'] = df_final['Reporting Country/ Territory/Area'].str.replace('\r', ' ', regex=False)

In [53]:
# output to csv so we can check if we're happy with the output
if not processed_data_folder.exists():
    processed_data_folder.mkdir()
    
df_final.to_csv(processed_data_folder / 'final_output.csv', index=False)

### Final notes

If the dataset isn't too large, I generally output to csv for a closer look. Once you've opened the file, you should see that we have a nice, clean output. Success!

In this notebook, we've gone through the full process of reading tables from 1 PDF, post-processing the data, and generating clean output. To generalize this into a reusable solution, I usually convert it to a python script. However, there is a library called [papermill](https://papermill.readthedocs.io/en/latest/) that can turn your notebook into an executable script. I've heard great things about it, but haven't tried it out myself. Something to do in one of my future projects 😃

I've also had a quick look at the other two reports in the raw data folder, and discovered the following:
- Report 72 was clean. All I needed to do, was adjust the page numbers and rerun all cells.
- Report 73 has a different problem with the data that requires additional steps to clean it, which again illustrates how hard it can be to work with PDFs. I've left that as an exercise for another time 😉
   
Thank you for making it all the way to the end! If you have any questions or comments, feel free to reach out to me. You can find my contact in the README.